In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [137]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
cur.execute("SELECT rowid, title FROM Korean_book WHERE title==\'우리의 일상을 지배하는 IT 거인들\'")
check = cur.fetchall()
print(check)
conn.commit()
conn.close()


[(15638, '우리의 일상을 지배하는 IT 거인들')]


In [143]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
cur.execute("SELECT title, author FROM Korean_book")
data = cur.fetchall()
print(len(data))
conn.commit()
conn.close()


44945


In [144]:
rdd = sc.parallelize(data)
print(rdd.take(2))
print(rdd.count())

[('하얼빈', '김훈'), ('아버지의 해방일지', '정지아')]
44945


In [145]:
S = set({})
D = set({})
for d in rdd.collect():
    if d in S:
        D.add(d[0])
    else:
        S.add(d)

print(len(S))
print(len(D)) # <--- 실제 중복 데이터(책 제목과 글쓴이도 다른)

44945
0


In [146]:
print(list(D)[0])
real_dup_data = list(D)
print(len(real_dup_data))

IndexError: list index out of range

In [142]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()

# 하나 빼고 중복되는 도서들 모두 삭제
for i in range(len(real_dup_data)):
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()
    del_id = []
    for num, (item) in enumerate(same):
        if num==0:
            pass
        else:
            del_id.append(item[0])

    for id in del_id:
        cur.execute(f"DELETE FROM Korean_book WHERE rowid={id}")
    conn.commit()
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()

conn.close()

In [147]:
## 중복 도서 삭제 확인
S = set({})
D = set({})
for d in rdd.collect():
    if d in S:
        D.add(d[0])
    else:
        S.add(d)

print(len(S))
print(len(D)) # <--- 실제 중복 데이터(책 제목과 글쓴이도 다른)

44945
0


In [60]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
cur.execute("SELECT title FROM Korean_book")
data1 = cur.fetchall()
print(len(data1))
conn.commit()
conn.close()

61185


In [61]:
rdd = sc.parallelize(data).map(lambda line: line[0])
print(rdd.take(2))
print(rdd.count())

['하얼빈', '아버지의 해방일지']
61185


In [62]:
S1 = set({})
D1 = set({})
for title in rdd.collect():
    if title in S1:
        D1.add(title)
    else:
        S1.add(title)

print(len(S1))
print(len(D1)) # <--- 책 이름만 같은 중복 데이터

44616
12631


In [72]:
same_title_but_diff_author = set({})
for d in D1:
    if d in D:
        pass
    else:
        same_title_but_diff_author.add(d)

print(len(same_title_but_diff_author))

270


In [71]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
cur.execute("SELECT * FROM Korean_book WHERE title=\"외식산업의 이해\"")
same = cur.fetchall()
for i in same:
    print(i)
    print()
conn.commit()
conn.close()

(21377, '외식산업의 이해', '국내도서,경제/경영,관광/호텔,호텔,외식/식음료,대학교재', '김관식 외', None, '양림출판사', 20190830, 359, '▶ 이 책은 외식사업을 다룬 이론서입니다.', 'https://contents.kyobobook.co.kr/sih/fit-in/142x0/pdt/9791189369125.jpg')

(21751, '외식산업의 이해', '국내도서,경제/경영,관광/호텔,호텔,외식/식음료,대학교재', '나정기', 7.5, '백산출판사', 20130210, 661, '『외식산업의 이해』는 외식산업의 개요를 시작으로 외식업체 운영의 문제점과 실패요인까지 총 19장으로 구성된 책이다. 외식산업과 관련된 산업의 이해 및 발전배경, 현황과 전망, 분류 등을 정리할 수 있다.', 'https://contents.kyobobook.co.kr/sih/fit-in/142x0/pdt/9788977392267.jpg')



In [4]:
temp = sc.parallelize([('이재정', '박희윤'), ("김민형", "이재정"), ("이재정", "박희윤")])
temp.take(1)
temp1 = temp.distinct()
for i in temp1.collect():
    print(i)


박희윤
김민형
이상원
이재정


In [34]:
s = set({('이재정', '박희윤')})
x = ('이재정', '박희윤')
print(s)
print(x)
if x in s:
    print("already")
else:
    print("not in set")

{('이재정', '박희윤')}
('이재정', '박희윤')
already
